# Getting the list of all .xml file names from Scisumm

In [1]:
!wget  https://cs.stanford.edu/~myasu/projects/scisumm_net/scisummnet_release1.1__20190413.zip

--2024-12-15 17:55:15--  https://cs.stanford.edu/~myasu/projects/scisumm_net/scisummnet_release1.1__20190413.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19451729 (19M) [application/zip]
Saving to: ‘scisummnet_release1.1__20190413.zip’

scisummnet_release1 100%[===================>]  18.55M   415KB/s    in 66s     

2024-12-15 17:56:23 (287 KB/s) - ‘scisummnet_release1.1__20190413.zip’ saved [19451729/19451729]



In [4]:
!unzip scisummnet_release1.1__20190413.zip

Archive:  scisummnet_release1.1__20190413.zip
   creating: scisummnet_release1.1__20190413/
  inflating: scisummnet_release1.1__20190413/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/scisummnet_release1.1__20190413/
  inflating: __MACOSX/scisummnet_release1.1__20190413/._.DS_Store  
  inflating: scisummnet_release1.1__20190413/Dataset_Documentation.txt  
  inflating: __MACOSX/scisummnet_release1.1__20190413/._Dataset_Documentation.txt  
  inflating: scisummnet_release1.1__20190413/log.txt  
  inflating: __MACOSX/scisummnet_release1.1__20190413/._log.txt  
   creating: scisummnet_release1.1__20190413/top1000_complete/
   creating: scisummnet_release1.1__20190413/top1000_complete/P99-1008/
  inflating: scisummnet_release1.1__20190413/top1000_complete/P99-1008/citing_sentences_annotated.json  
   creating: __MACOSX/scisummnet_release1.1__20190413/top1000_complete/
   creating: __MACOSX/scisummnet_release1.1__20190413/top1000_complete/P99-1008/
  inflating: __MACOSX/scisummnet_r

In [5]:
%%time 

from lxml import objectify 
import pandas as pd 
import numpy as np 
import os 
import glob 
from glob import iglob 


CPU times: user 597 ms, sys: 866 ms, total: 1.46 s
Wall time: 689 ms


In [12]:
# Create xml file extraction function 

def exract_xml(directory):
    xml_data = objectify.parse(directory)
    root = xml_data.getroot() # Root element 

    data = []
    cols = []

    for i in range(len(root.getchildren())):
        child = root.getchildren()[i]
        data.append([subchild.text for subchild in child.getchildren()])
        # If the tag is not 'SECTION', it is a section header, append that header
        # If it is, it means it is a subsection, and append the title of that subsection
        if child.tag != "SECTION":
            cols.append(child.tag)
        else :
            cols.append(child.attrib.get('title'))    

    df = pd.DataFrame(data).T 
    df.columns = cols # update column names 

    # Get the abstract column (second column)
    abstract_list = df.iloc[:, 1].dropna()
    abstract = " ".join(abstract_list)

    # Get the conclusion column ()
    conclusion_text = df.iloc[:, -2].dropna()
    conclusion = " ".join(conclusion_text)

    # Drop last column of a dataframe
    df = df.iloc[: , :-1]

    # Drop first column: S 
    df = df.iloc[:, 1:]

    # Iterate over all sections and join them together tpo get the text document 
    text_list = []
    for column in df.columns:
        text_filtered = df[column].dropna()
        text = " ".join(text_filtered)
        text_list.append(text)

    text_list    
    final_text = " ".join(text_list)

    return abstract, final_text, conclusion


In [9]:
print(os.getcwd())  # Prints the current working directory


/home/yassine/Textra-edu/data


In [ ]:
import glob

file_directory = glob.glob("scisummnet_release1.1__20190413/**/*.xml", recursive=True)
print(file_directory[:5])  # Show the first 5 matching files


['scisummnet_release1.1__20190413/top1000_complete/W04-3219/Documents_xml/W04-3219.xml', 'scisummnet_release1.1__20190413/top1000_complete/J03-3002/Documents_xml/J03-3002.xml', 'scisummnet_release1.1__20190413/top1000_complete/J96-1002/Documents_xml/J96-1002.xml', 'scisummnet_release1.1__20190413/top1000_complete/P08-1030/Documents_xml/P08-1030.xml', 'scisummnet_release1.1__20190413/top1000_complete/N01-1006/Documents_xml/N01-1006.xml']


In [13]:
# Create lists for abstract, full text, and conclusion
abstract_list = []
full_text_list = []
conclusion_list = []

counter = 0 

for directory in file_directory:
    abstract, full_text, conclusion = exract_xml(directory)
    abstract_list.append(abstract)
    full_text_list.append(full_text)
    conclusion_list.append(conclusion)

    print(f"XML extraction for document {counter} done!")
    counter +=1

text_df = pd.DataFrame(list(zip(abstract_list, full_text_list, conclusion_list)), columns=["abstract", "full_text", "conclusion"])    
print(text_df.head())


# Save to CSV file 
text_df.to_csv("top1000.csv")

XML extraction for document 0 done!
XML extraction for document 1 done!
XML extraction for document 2 done!
XML extraction for document 3 done!
XML extraction for document 4 done!
XML extraction for document 5 done!
XML extraction for document 6 done!
XML extraction for document 7 done!
XML extraction for document 8 done!
XML extraction for document 9 done!
XML extraction for document 10 done!
XML extraction for document 11 done!
XML extraction for document 12 done!
XML extraction for document 13 done!
XML extraction for document 14 done!
XML extraction for document 15 done!
XML extraction for document 16 done!
XML extraction for document 17 done!
XML extraction for document 18 done!
XML extraction for document 19 done!
XML extraction for document 20 done!
XML extraction for document 21 done!
XML extraction for document 22 done!
XML extraction for document 23 done!
XML extraction for document 24 done!
XML extraction for document 25 done!
XML extraction for document 26 done!
XML extract

In [15]:
text_df[0:5]

,abstract,full_text,conclusion
0,We apply statistical machine translation (SMT)...,We apply statistical machine translation (SMT)...,We presented a novel approach to the problem o...
1,Parallel corpora have become an essential reso...,Parallel corpora have become an essential reso...,"For each item, participants were instructed to..."
2,The concept of maximum entropy can be traced b...,The concept of maximum entropy can be traced b...,We began by introducing the building blocks of...
3,We apply the hypothesis of “One Sense Per Disc...,We apply the hypothesis of “One Sense Per Disc...,The trigger labeling task described in this pa...
4,Transformation-based learning has been success...,Transformation-based learning has been success...,We have presented in this paper a new and impr...
